In [48]:
import json, requests

s = requests.session() # establish the session since we'll be making multiple requests
# my id 5842359
def authorize_user(session, client_id, redirect_uri = None):
    if redirect_uri == None:
        redirect_uri = 'https://oauth.vk.com/blank.html'
    oauth = session.head('https://oauth.vk.com/authorize', params = {'client_id' : str(client_id), 
                                                                    'redirect_uri' : redirect_uri,
                                                                    'display' : 'page',
                                                                    'scope' : 'friends',
                                                                    'response_type' : 'code',
                                                                    'v' : 5.63},
                       allow_redirects = True)
    print oauth.history

authorize_user(s, 1)

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

In [17]:
token = 'bda4423f6da44fb99cd1fdf4cc606bfed2f7b8eb1919912b40daf15547270d86538052eb61ed5f7da27ec'

req_friends = 'https://api.vk.com/method/friends.get?v=5.52&access_token=' + token

params = {'fields' : ['first_name', 'last_name']}
resp_friends = requests.get(req_friends, params = params)
friends = json.loads(resp_friends.text)['response']['items']
nodes = ['Galina Baimeeva']
ids = [146062456]

for i in friends:
    nodes.append(i['first_name'].encode('utf8') + ' ' + i['last_name'].encode('utf8'))
    ids.append(i['id'])

ids_dict = {key : value for (value, key) in zip(range(len(ids)), ids)}
nodes_dict = {key : value for (key, value) in zip(ids, nodes)}
labels_dict = {key: value for (key, value) in zip(range(len(ids)), nodes)}
mutuals = []
edges = []

def execute(ids, start):
    code = {'code' : 'return ['}
    for i in range(start, min(start + 25, len(ids))):
        code['code'] += 'API.friends.getMutual({"source_uid" : ' + str(ids[0]) + ', "target_uid": ' + str(ids[i]) + '}), '    
    code['code'] += '];'
    req = 'https://api.vk.com/method/execute?v=5.52&access_token=' + token
    r = requests.get(req, params = code)
    response = json.loads(r.text)['response']
    resp_dict = {key : value for (key, value) in zip(ids[start:(start + 25)], response)}
    return resp_dict

def add_mutuals(edges, i):
    mutuals = execute(ids, i)
    for j in mutuals:
        for k in mutuals[j]:
            edges.append((nodes_dict[j], nodes_dict[k]))

for i in range(0, len(nodes), 25):
    add_mutuals(edges, i)

KeyError: 'response'

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

k = nx.degree(G)
plt.figure(1, figsize=(15, 15))
coord = nx.spring_layout(G)
nx.draw_networkx(G,
                 pos=coord,
                 nodelist=k.keys(),
                 node_size = [(d + 1)*10 for d in k.values()],
                 node_color=k.values(),
                 font_size=8,
                 cmap=plt.cm.Reds,
                 )